In [5]:
import json, zstandard as zstd, pandas as pd, numpy as np, requests, io, os
from datetime import datetime, timedelta

In [8]:
zst_path = "./glbx-mdp3-20201009-20251008.ohlcv-1d.csv.zst"

with open(zst_path, "rb") as fh:
    dctx = zstd.ZstdDecompressor()
    with dctx.stream_reader(fh) as reader:
        df = pd.read_csv(reader, encoding="utf-8", parse_dates=["ts_event"])

df.head(50)

,ts_event,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol
0,2020-10-09 00:00:00+00:00,35,1,178892,-0.0097,-0.0093,-0.0101,-0.0096,439,HOJ1-HOK1
1,2020-10-09 00:00:00+00:00,35,1,112388,-0.0800,-0.0700,-0.0800,-0.0700,10,CLG2-CLJ2
2,2020-10-09 00:00:00+00:00,35,1,93394,-0.0800,-0.0600,-0.1100,-0.0800,961,CLZ1-CLH2
3,2020-10-09 00:00:00+00:00,35,1,4135,-3.8600,-3.8600,-3.8600,-3.8600,1,CLZ0-BZM1
4,2020-10-09 00:00:00+00:00,35,1,180969,-0.0135,-0.0132,-0.0151,-0.0135,577,RBJ1-RBK1
5,2020-10-09 00:00:00+00:00,35,1,26650,-0.1300,-0.1300,-0.1500,-0.1500,1476,CLM1-CLN1
6,2020-10-09 00:00:00+00:00,35,1,289790,-0.0136,-0.0122,-0.0137,-0.0125,7529,HOZ0-HOF1
7,2020-10-09 00:00:00+00:00,35,1,245031,-0.0026,-0.0016,-0.0026,-0.0021,76,HO:BF F1-G1-H1
8,2020-10-09 00:00:00+00:00,35,1,152252,-0.0044,-0.0038,-0.0044,-0.0039,59,HO:BF G1-H1-J1
9,2020-10-09 00:00:00+00:00,35,1,100895,-0.3300,-0.3100,-0.3900,-0.3300,1044,CLM2-CLZ2


In [9]:
#clean
df = df.drop(columns=['publisher_id', 'rtype'])
df['ts_event'] = df['ts_event'].dt.date          
df = df.rename(columns={'ts_event': 'date'})

In [10]:
df.head()
len(df)

541776

In [11]:
df['date'] = pd.to_datetime(df['date'])          
df = df[df['date'].dt.to_period('M') != '2025-04'] #remove liberation day data

In [12]:
len(df)

531311

In [13]:
CL  = df[df['symbol'].str.match(r'^CL[A-Z]\d-CL[A-Z]\d$')].copy()

In [14]:
len(CL)

168753

In [15]:
CL.reset_index(drop=True, inplace=True)

In [16]:
CL = CL[CL['volume'] > 100]

In [17]:
len(CL)
CL.head(20)

,date,instrument_id,open,high,low,close,volume,symbol
1,2020-10-09,93394,-0.08,-0.06,-0.11,-0.08,961,CLZ1-CLH2
2,2020-10-09,26650,-0.13,-0.13,-0.15,-0.15,1476,CLM1-CLN1
3,2020-10-09,100895,-0.33,-0.31,-0.39,-0.33,1044,CLM2-CLZ2
4,2020-10-09,190323,-0.74,-0.72,-0.84,-0.78,482,CLG1-CLK1
8,2020-10-09,220882,-0.90,-0.89,-1.01,-0.99,6006,CLZ0-CLH1
9,2020-10-09,29711,-0.90,-0.88,-1.01,-1.00,4056,CLX0-CLG1
10,2020-10-09,57124,-0.32,-0.31,-0.37,-0.35,2061,CLM1-CLU1
11,2020-10-09,35004,-0.48,-0.41,-0.57,-0.46,1355,CLZ2-CLZ3
13,2020-10-09,26488,-0.77,-0.74,-0.88,-0.82,354,CLH1-CLN1
15,2020-10-09,63802,-0.19,-0.17,-0.23,-0.21,838,CLU1-CLZ1


In [18]:
CL.reset_index(drop=True, inplace=True)